# Klasyfikator klastrów

## IMPORTS

In [ ]:
!pip install comet-ml

In [1]:
import comet_ml

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import einops
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# import os

# from google.colab import userdata
# key = userdata.get('COMET_API_KEY')
# os.environ['COMET_API_KEY'] = key

In [2]:
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # Set default tensor type for cuda
        torch.set_default_dtype(torch.float32)
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
        # Ensure we're using float32 on CPU
        torch.set_default_dtype(torch.float64)
    return device

device = setup_device()

print(f"Using {device} device")

Using cuda device


## Dataset

In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset, DatasetDict, load_from_disk


In [ ]:
!ls

In [4]:
ds = load_from_disk("clustered_dataset").with_format("torch")

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 82600
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 10325
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 10325
    })
})

In [ ]:
# from datasets import DatasetDict
# 
# # Create subsets for each cluster
# cluster_subsets = {}
# 
# for cluster_id in range(0,20):
#     # Filter the dataset for the current cluster
#     print(int(cluster_id))
#     cluster_data = ds.filter(lambda example: example['cluster'] == cluster_id)
#     
#     # Add to the cluster_subsets dictionary
#     cluster_subsets[int(cluster_id)] = cluster_data
# 
# # cluster_subsets now contains subsets for 20 clusters

In [6]:
from datasets import load_from_disk

# Initialize the dictionary to store loaded subsets
cluster_subsets = {}

# Iterate over the range of cluster IDs (assuming 20 clusters)
for cluster_id in range(0, 20):
    # Construct the path to the saved subset
    subset_path = f"clusters/cluster_{cluster_id}_subset"
    
    # Load the subset from disk and add it to the dictionary
    cluster_subsets[cluster_id] = load_from_disk(subset_path).with_format("torch")

In [7]:
print(cluster_subsets)

{0: DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 2904
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 380
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 389
    })
}), 1: DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 3044
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 354
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'cluster'],
        num_rows: 

In [ ]:
# for cluster_id, subset in cluster_subsets.items():
#     subset.set_format("arrow")
#     subset.save_to_disk(f"clusters/cluster_{cluster_id}_subset")
#     subset.set_format("torch")
    

In [8]:
# change if not colab
num_workers = 12
pin_memory = True
batch_size = 32

cluster_dataloaders = {}

for cluster_id in range(0,20):
    subset = cluster_subsets[cluster_id]
    cluster_dataloaders[cluster_id] = {
            "train": DataLoader(
                subset["train"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=True,
                pin_memory=pin_memory,
            ),
            "test": DataLoader(
                subset["test"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=False,
                pin_memory=pin_memory,
            ),
            "valid": DataLoader(
                subset["valid"],
                batch_size=batch_size,
                num_workers=num_workers,
                shuffle=False,
                pin_memory=pin_memory,
            ),
        }



In [9]:
print(cluster_dataloaders)

{0: {'train': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1A4690>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE16C990>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x00000262C8FAAF10>}, 1: {'train': <torch.utils.data.dataloader.DataLoader object at 0x00000262D2383310>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F6410>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F64D0>}, 2: {'train': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F7050>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F7290>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F74D0>}, 3: {'train': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F76D0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F7790>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x00000262FE1F7850>}, 4: {'train': <torch.uti

## AUTOENCODER

In [10]:
from torchvision import models

class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        self.feature_extractor = models.vgg16(pretrained=True).features.eval()
        self.layers = ['4', '9', '16']
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

    def forward(self, predicted, targets):
        pred_features = self.extract_features(predicted)
        target_features = self.extract_features(targets)

        loss = 0.0
        for pred_feat, target_feat in zip(pred_features, target_features):
            loss += nn.functional.mse_loss(pred_feat, target_feat)
        return loss

    def extract_features(self, x):
        features = []
        for name, layer in self.feature_extractor._modules.items():
            x = layer(x)
            if name in self.layers:
                features.append(x)
        return features


class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.perceptual_loss = PerceptualLoss()
        self.mse_loss = nn.MSELoss(reduction='none')
        self.alpha = alpha

    def forward(self, predicted, targets, masks):
        loss_weights = torch.where(masks == 0.0, torch.tensor(1.0), torch.tensor(10.0))
        loss_weights = loss_weights.unsqueeze(1)  # batch 1 h w
        loss_weights = loss_weights.to(predicted.device)


        mse = self.mse_loss(predicted, targets) * loss_weights
        mse = mse.mean()

        perceptual = self.perceptual_loss(predicted, targets)

        return self.alpha * mse + (1 - self.alpha) * perceptual


In [11]:
from torchinfo import summary
from tqdm import tqdm 
from create_mask import generate_scaled_blob

LR = 3e-5
num_epochs = 2
loss_func = CombinedLoss(alpha=0.7).to(device)





C:\Users\kamil\.conda\envs\DLF\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kamil\.conda\envs\DLF\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
from create_mask import generate_scaled_blob
import torch, gc

for cluster in range(0,20):
    
    PATH = "models/inpating/mse_perceptual/model1.pth"
    # model = VGG16Autoencoder()
    model = torch.load(PATH)
    model.eval()
    model.to(device)
    
    train_loader = cluster_dataloaders[cluster]['train']
    test_loader= cluster_dataloaders[cluster]['test']
    val_loader = cluster_dataloaders[cluster]['valid']
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    # Train and validate
    for epoch in range(num_epochs):

        # Training phase
        model.train()

        for idx, batch in tqdm(enumerate(train_loader), desc=f"TRAIN_{epoch}"):
            optimizer.zero_grad()  # Zero out gradients before each batch

            images = batch["image"].float() / 255.0
            labels = batch["style"]  # Assuming you still have this in your dataset

            # masks with corrupted part of image
            masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0

            # apply mask
            images_with_mask = images * (1 - masks.unsqueeze(1))

            # add mask as 4th channel
            images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1)  # batch 4 imgsize imgsize

            inputs = images_with_mask.to(device)
            targets = images.to(device)

            reconstructed_image = model(inputs)
            loss = loss_func(reconstructed_image, targets, masks)
            loss.backward()
            optimizer.step()
            if idx % 25 == 0:
                print(f"Train Model {cluster}  epoch {epoch} batch {idx} loss: {loss.item()}")
        
        # Validation phase
        model.eval()
        with torch.no_grad():
            for idx, batch in tqdm(enumerate(val_loader), desc=f"VAL_{epoch}"):
    
                images = batch["image"].float() / 255.0
    
                # masks with corrupted part of image
                masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0
    
                #apply mask
                images_with_mask = images * (1- masks.unsqueeze(1))
    
                #add mask as 4th channel
                images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1) # batch 4 imgsize imgsize
    
                inputs = images_with_mask.to(device)
                targets = images.to(device)
    
                # Forward pass through the model
                # Autoencoder directly outputs the reconstructed image
                reconstructed_image = model(inputs)
    
                loss = loss_func(reconstructed_image, targets, masks)
                
                if idx % 25 == 0:
                    print(f"VAl Model {cluster}  epoch {epoch} batch {idx} loss: {loss.item()}")


    # Set the model to evaluation mode
    model.eval()

    print("Start test")
    # Begin the test phase
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(test_loader), desc=f"TEST_{num_epochs}"):
    
            # Load and preprocess images
            images = batch["image"].float() / 255.0  # Assuming images are in [0, 255], normalize to [0, 1]
            labels = batch["style"]  # Assuming you still have this in your dataset (though not used here)
    
            # masks with corrupted part of image
            masks = generate_scaled_blob(images.shape, mask_percentage=(1 / 16) * 100).float() / 255.0
    
            # apply mask
            images_with_mask = images * (1 - masks.unsqueeze(1))
    
            # add mask as 4th channel
            images_with_mask = torch.cat((images_with_mask, masks.unsqueeze(1)), dim=1)  # batch 4 imgsize imgsize
            
            inputs = images_with_mask.to(device)
            targets = images.to(device)
            
            reconstructed_image = model(inputs)
    
            loss = loss_func(reconstructed_image, targets, masks)
            
            if idx % 25 == 0:
                print(f"Test Model {cluster}  batch {idx} loss: {loss.item()}")      
        
        
    print("Save model dict")
    torch.save(model.state_dict(), f"models/inpating/submodels/cluster_{cluster}_submodel_dict.pth")
    print("Save model")
    torch.save(model, f"./cluster_{cluster}_submodel.pth")
    
    del model
    torch.cuda.empty_cache() 
    gc.collect()
            


C:\Users\kamil\AppData\Local\Temp\ipykernel_11728\2872865626.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(PATH)
TRAIN_0: 2it [00:04,  1.89s/it]

Train Model 0  epoch 0 batch 0 loss: 0.12263540923595428


TRAIN_0: 5it [00:05,  1.41it/s]C:\Users\kamil\.conda\envs\DLF\Lib\site-packages\scipy\interpolate\_fitpack_py.py:155: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  res = _impl.splprep(x, w, u, ub, ue, k, task, s, t, full_output, nest, per,
TRAIN_0: 27it [00:14,  2.68it/s]

Train Model 0  epoch 0 batch 25 loss: 0.11066010594367981


TRAIN_0: 52it [00:24,  2.69it/s]

...

Start test


TEST_2: 2it [00:03,  1.36s/it]

Test Model 19  batch 0 loss: 0.149053156375885


TEST_2: 10it [00:05,  1.86it/s]


Save model dict
Save model
